Import Libraries and updates

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle JSON files
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
!conda install -c conda-forge beautifulsoup4 --yes
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Block1: Success')

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Block2: Success')


# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Block3: Success')

!conda install -c conda-forge folium=0.5.0 --yes
#import folium # map rendering library
!pip install folium

!conda update -n base -c defaults conda 
print('Block4: Success')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Block1: Success
Block2: Success
Block3: Success
Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    chardet-3.0.4              |        py37_1003         174 KB
    ------------------------------------------------------------
                                           Total:         174 KB

The following packages will be REMOVED:

  _libgcc_mutex-0.1-main
  conda-env-2.6.0-1

The following packages will be UPDATED:

  chardet                                      3.0.4-py37_1 --> 3.0.4-py37_1003
  libstdcxx-ng                             8.2.0-hdf63c60_1 --> 9.

WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /opt/conda/pkgs/chardet-3.0.4-py37_1003/lib/python3.7/site-packages/chardet/escprober.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /opt/conda/pkgs/chardet-3.0.4-py37_1003/lib/python3.7/site-packages/chardet/langthaimodel.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /opt/conda/pkgs/chardet-3.0.4-py37_1003/lib/python3.7/site-packages/chardet/jpcntx.py.  Please remove this file manually (you may need to reboot to free file handles)
WARNING conda.gateways.disk.delete:unlink_or_rename_to_trash(140): Could not remove or rename /opt/conda/pkgs/chardet-3.0.4-py37_1003/lib/python3.7/site-packages/chardet/gb2312freq.py.  Please remove this fi

Block4: Success


Read Wiki Table

In [5]:
# GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#url = ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#url = ('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695')
#results = requests.get(url).json
#results


In [6]:
soup = BeautifulSoup(data, 'html.parser')

Create Lists

In [31]:
PostalCodeList1 = []
BoroughList1 = []
NeighborhoodList1 = []

Read Table

In [33]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [34]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCodeList1.append(cells[0].text.rstrip('\n'))
        BoroughList1.append(cells[1].text.rstrip('\n'))
        NeighborhoodList1.append(cells[2].text.rstrip('\n'))

Create Dataframe

In [36]:
toronto_df1 = pd.DataFrame({"PostalCode": PostalCodeList1,
                           "Borough": BoroughList1,
                           "Neighborhood": NeighborhoodList1})

toronto_df1.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Ignore cells with a borough that is Not assigned

In [41]:
toronto_df_drop1 = toronto_df1[toronto_df1.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Combine Neigborhoods

In [43]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_drop1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Assign Null Neighborhood Borough Values

In [44]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Testing

Print Shape

In [45]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)